In [ ]:
# Standard data manipulation libraries
import pandas as pd
import numpy as np

# Webscraping imports
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# Test with random sample of 1000 ssvids
sample_ssvid = pd.read_csv('./data/sample_ssvid.csv')

In [ ]:
ssvids = sample_ssvid[['ssvid']]

In [ ]:
# convert ssvid column to list of numbers
ssvids = sample_ssvid['ssvid'].values.tolist()
ssvids[:5]

## Algorithm 1

In [ ]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()


# Create a function that searches for each MMSI number in the IMO database
# and classifies them into registered/not registered lists 

imo_reg = []
imo_notreg = []

def check_reg(num):
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    time.sleep(1)
    #table = driver.find_element_by_id('ctl00_bodyPlaceHolder_Default_gridShips')
    #table = driver.find_element_by_class_name("gridviewer_grid")
    table = driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]')
    rows = table.find_elements(By.TAG_NAME, "tr")
    if len(rows) <= 1:
        imo_notreg.append(num)
    elif len(rows) >= 2:
        imo_reg.append(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear() 
    return None

# Iterate through the list of MMSI numbers
for num in ssvids:
    check_reg(num)
print(len(imo_reg), len(imo_notreg))

In [ ]:
# previously 3
len(imo_reg)

In [ ]:
# previously 107
len(imo_notreg)

## Algorithm 2

- Added second if statement to catch multiple rows.
- The header of the table is also being counted as a row - increased len of if statements. One search result will have len of 2.
- used code from previous work where successful on scraping one search result
- Only remaining issue is skipping the first row of multiple search results

In [ ]:
# test MMSI num with one result and one with multiple
test = [23510249, 900]

In [ ]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]

# For each number in ssvids
for num in test:
    
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(5)
    
    # define the results table and rows
    table = driver.find_element_by_id('ctl00_bodyPlaceHolder_Default_gridShips')
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    # if there is only one search result
    if len(rows) == 2:
        
        # Find and click the result
        driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr').click()
#         WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
#         driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{row}]').click()
#         driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/td').click()
        
        # scrape and append the data into lists
        name.append(driver.find_element_by_id("sNameCur").text)
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
        flag.append(driver.find_element_by_id("sFlagCur").text)
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
        vessel_type.append(driver.find_element_by_id("sTypeCur").text)
        built.append(driver.find_element_by_id("sBuildDateCur").text)
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
        owner.append(driver.find_element_by_id("sRegOwnerCur").text)
        time.sleep(1)
        
        # return to the search results page and clear search field
        driver.execute_script("window.history.go(-1)")
        
        # Clear search field    
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
        
    # if there are multiple search results, loop through each result
    elif len(rows) > 2:
        for row in range(1, len(rows)+1):

            # wait until element appears, then click it
            WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
            driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{row}]').click()

            # scrape and append the data into lists
            name.append(driver.find_element_by_id("sNameCur").text)
            imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
            flag.append(driver.find_element_by_id("sFlagCur").text)
            call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
            mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
            vessel_type.append(driver.find_element_by_id("sTypeCur").text)
            built.append(driver.find_element_by_id("sBuildDateCur").text)
            tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
            owner.append(driver.find_element_by_id("sRegOwnerCur").text)
            time.sleep(1)

            # return to the search results page to scrape next result
            driver.execute_script("window.history.go(-1)")

    # Clear search field
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

In [ ]:
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner

In [ ]:
# transform lists into pandas dataframe
imo = pd.DataFrame({'name': name,
                     'imo_num': imo_num,
                     'flag': flag,
                     'call_sign': call_sign,
                     'mmsi': mmsi,
                     'vessel_type': vessel_type,
                     'built' : built,
                     'tonnage': tonnage,
                     'owner': owner})

## References
- https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
- https://selenium-python.readthedocs.io/locating-elements.html
- https://stackoverflow.com/questions/27338742/how-do-i-send-a-delete-keystroke-to-a-text-field-using-selenium-with-python
- https://stackoverflow.com/questions/12057118/iterate-through-table-in-selenium-2-webdriver-python